In [ ]:
# ref_github: https://github.com/StanislavLukashevich/LinkedIn-Python-Content-Analytics/blob/master/LinkedIn_WebScraping_Content_Performance_Analytics.py

In [42]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import getpass
from datetime import datetime
import pandas as pd

In [ ]:
#get User Informatio
print("Enter Username")
username_cred =  str(input())
print("Enter Password")
password_cred = getpass.getpass("Enter Password: ")
print("How many posts are you interested in?")
post_count = int(input())

print("********Details********")
print(f"Username: {username_cred}")
print("Password:", "*" * len(password_cred))
print(f"Count: {post_count}")

#create a browser-specific (Google Chrome) web navigation simulator
#download from https://googlechromelabs.github.io/chrome-for-testing/ and download chromedrove.exe that matches your chrome version
#to check your chrome verison open chrome -> settings -> about chrome
#save this driver in the python env directory
browser = webdriver.Chrome("C://Users//ashis//OneDrive//Desktop//MG Sustainable//02. HeatWise//WebScrape_LinkedIn//chromedriver.exe.")

#opens the LinkedIn login page and login under a specified account:
browser.get('https://www.linkedin.com/login')

#enter the specified information to login to LinkedIn:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username_cred)
time.sleep(5)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password_cred)
time.sleep(5)

elementID.submit()

#direct to HEATWISE's linkedin page
activity_link = "https://www.linkedin.com/company/heatwise-horizon-europe-project-2023/posts/?feedView=all" 
time.sleep(2)
browser.get(activity_link)

#calculate number of scrolls depending on the input
number_of_scrolls = -(-post_count // 5)  # 5 is LinkedIn's number of posts per scroll based on screen size

#we need a loop because we have a particular number of scrolls...
likes = []
comments = []

SCROLL_PAUSE_TIME = 5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

for scroll in range(number_of_scrolls) : 
    #scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    #calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height    

#query the contents (returns service reponse object with web contents, url headers, status and other):
src = browser.page_source
#beautiful soup instance:
soup = BeautifulSoup(src, features="lxml")   #lxml
    
#find LIKES on LinkedIn
#look for "span" tags that have the specific following attribute (click 'inspect' on the L-in page)
#need to convert the list of bs4 tags into strings and then extract 
#find these specific tags ("<stuff>") in the soup contents:

likes_bs4tags = soup.find_all("span", attrs = {"class" : "social-details-social-counts__reactions-count"})
#converts a list of 1 string to int, appends to likes list
for tag in likes_bs4tags:
    strtag = str(tag)
    #the first argument in findall (below) is a regular expression (accounts for commas in the number)
    list_of_matches = re.findall('[,0-9]+',strtag)
    #converts the last element (string) in the list to int, appends to likes list
    last_string = list_of_matches.pop()
    without_comma = last_string.replace(',','')
    likes_int = int(without_comma)
    likes.append(likes_int)

comments_bs4tags = soup.find_all("li", class_="social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--right-aligned")

for tag in comments_bs4tags:
    #find the button tag within the list item
    button_tag = tag.find("button")
    
    #check if the button tag and aria-label exist
    if button_tag and 'aria-label' in button_tag.attrs:
        aria_label = button_tag['aria-label']
        print(aria_label)
        
        #extract the number from the aria-label
        comments_count = int(re.search(r'\d+', aria_label).group())
    else:
        #default to 0 if not found
        comments_count = 0
    
    #a the count to the list
    comments.append(comments_count)

#calculate total likes and comments
total_likes = sum(likes)
total_comms = sum(comments)

new_row = {
    'no_likes': total_likes,
    'no_comments': total_comms,
    'date': datetime.today().date()
}

#create a DataFrame with the new row
new_df = pd.DataFrame([new_row])

df = pd.read_excel("raw_data.xlsx")

df = pd.concat([df, new_df], ignore_index=True)
df

In [52]:
#save as CSV or xlsx
df.to_excel('raw_data.xlsx', index=False)